In [1]:
import numpy as np
import h5py

import sys
sys.path.append('../src')
from hdf_dataset import HDF3DIterator

In [2]:
arr = np.arange(20*500*32).reshape(20,500, 32)
arr

array([[[     0,      1,      2, ...,     29,     30,     31],
        [    32,     33,     34, ...,     61,     62,     63],
        [    64,     65,     66, ...,     93,     94,     95],
        ...,
        [ 15904,  15905,  15906, ...,  15933,  15934,  15935],
        [ 15936,  15937,  15938, ...,  15965,  15966,  15967],
        [ 15968,  15969,  15970, ...,  15997,  15998,  15999]],

       [[ 16000,  16001,  16002, ...,  16029,  16030,  16031],
        [ 16032,  16033,  16034, ...,  16061,  16062,  16063],
        [ 16064,  16065,  16066, ...,  16093,  16094,  16095],
        ...,
        [ 31904,  31905,  31906, ...,  31933,  31934,  31935],
        [ 31936,  31937,  31938, ...,  31965,  31966,  31967],
        [ 31968,  31969,  31970, ...,  31997,  31998,  31999]],

       [[ 32000,  32001,  32002, ...,  32029,  32030,  32031],
        [ 32032,  32033,  32034, ...,  32061,  32062,  32063],
        [ 32064,  32065,  32066, ...,  32093,  32094,  32095],
        ...,
        [ 47

In [3]:
with h5py.File('./test.h5', 'w') as f:
    f.create_dataset('data', data=arr)



In [5]:
with HDF3DIterator('./test.h5', 'data') as hdf:
    print(hdf[0])
    print(hdf[0:10])
    print(hdf[0,0])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28  29  30  31]
 [ 32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49
   50  51  52  53  54  55  56  57  58  59  60  61  62  63]
 [ 64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
   82  83  84  85  86  87  88  89  90  91  92  93  94  95]
 [ 96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113
  114 115 116 117 118 119 120 121 122 123 124 125 126 127]
 [128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
  146 147 148 149 150 151 152 153 154 155 156 157 158 159]
 [160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177
  178 179 180 181 182 183 184 185 186 187 188 189 190 191]
 [192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209
  210 211 212 213 214 215 216

In [6]:
with HDF3DIterator('./test.h5', 'data') as hdf:
    print(hdf[0])
    print(hdf[1])
    print(hdf[2])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
[32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55
 56 57 58 59 60 61 62 63]
[64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87
 88 89 90 91 92 93 94 95]


In [8]:
with HDF3DIterator('./test.h5', 'data') as hdf:
    for i, data in enumerate(hdf):
        print(i, data)
        if i > 10:
            break

0 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
1 [32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55
 56 57 58 59 60 61 62 63]
2 [64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87
 88 89 90 91 92 93 94 95]
3 [ 96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113
 114 115 116 117 118 119 120 121 122 123 124 125 126 127]
4 [128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159]
5 [160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177
 178 179 180 181 182 183 184 185 186 187 188 189 190 191]
6 [192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209
 210 211 212 213 214 215 216 217 218 219 220 221 222 223]
7 [224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241
 242 243 244 245 246 247 248 249 250 251 252 253 254 255]
8 [256 257 258 259 260 261 262 2